# Imports

In [1]:
!pip install python-crfsuite
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from itertools import chain

!pip install sklearn_crfsuite

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import copy


ne_path ="/content/drive/MyDrive/Colab Notebooks/project4_data/NE.txt"
ma_path = "/content/drive/MyDrive/Colab Notebooks/project4_data/NE.ma.txt"
path_loc =  "/content/drive/MyDrive/Colab Notebooks/project4_data/lexicon_location.txt"
path_per =  "/content/drive/MyDrive/Colab Notebooks/project4_data/lexicon_person.txt"
path_org =  "/content/drive/MyDrive/Colab Notebooks/project4_data/lexicon_organization.txt"

ne_deneme_path = "/content/drive/MyDrive/Colab Notebooks/project4_data/NE.prep.txt" 

     |████████████████████████████████| 747kB 4.2MB/s 


# Preparing Data

In [2]:
my_data = []
with open(ne_path) as f:
    for line in f:
      my_data.append(line[:-1])


my_data = []
with open(ne_path) as f:
    for line in f:
      my_data.append(line[:-1])

tagged_data = []
mode = 0 # 0->no tag, 1->beg_tag 2-> mid_tag
for i in range(len(my_data)):
  for each in my_data[i].split():
    if mode == 0: # no tag
      if each == "<b_enamex":
        mode = 1
      else:
        tagged_data.append(str(i+1) + " " + each + " O")
    elif mode == 1: # beg_tag
      tag_type = each.split("\"")[1]
      if "<e_enamex>" in each:
        tagged_data.append(str(i+1) + " " + each.split("\"")[2].split("<")[0][1:] + " B-" + tag_type[:3] )
        mode = 0
      else:
        tagged_data.append(str(i+1) + " " + each.split("\"")[2].split(">")[1] + " B-" + tag_type[:3] )
        mode = 2
    elif mode == 2: # mid_tag
      if "<e_enamex>" in each:
        tagged_data.append(str(i+1) + " " + each.split("<")[0] + " I-" + tag_type[:3] )
        mode = 0
      else:
        tagged_data.append(str(i+1) + " " + each + " I-" + tag_type[:3] )


ner_tagged_data = tagged_data
ner_tagged_list_of_sentences = []

line_cnt = 1
temp_line = []
for each in ner_tagged_data:
  current_line = int(each.split()[0])
  if current_line == line_cnt:
    temp_line.append(each)
  else:
    ner_tagged_list_of_sentences.append(temp_line)
    temp_line = [each]
    line_cnt += 1
ner_tagged_list_of_sentences.append(temp_line)


ma_data = []
with open(ma_path) as f:
    for line in f:
        ma_data.append(line[:-1])
ma_data_list_of_sentences = []

line_cnt = 1
temp_line = []
for each in ma_data:
  current_line = int(each.split()[0])
  if current_line == line_cnt:
    temp_line.append(each)
  else:
    ma_data_list_of_sentences.append(temp_line)
    temp_line = [each]
    line_cnt += 1
ma_data_list_of_sentences.append(temp_line)

df = []

for i in range(len(ner_tagged_list_of_sentences)):
    sentence = ner_tagged_list_of_sentences[i]
    ma_cnt = 0
    temp_sent = []
    for j in range(len(sentence)):
      word = sentence[j]
      if ma_cnt < len(ma_data_list_of_sentences[i])  and  word.split()[1] in ma_data_list_of_sentences[i][ma_cnt].split()[1]:
        ma_tag = ma_data_list_of_sentences[i][ma_cnt].split()[2]
        ma_cnt += 1
      else:
        ma_tag = "*UNKNOWN*"
      line_no = word.split()[0]
      token = word.split()[1]
      if(len(word.split()) < 3):
        ner_tag = word.split()[1]
      else:
        ner_tag = word.split()[2]

      temp_sent.append(line_no + " " + token + " " + ma_tag + " " + ner_tag)
    df.append(temp_sent)
  
ner_data_sentenced = df

to_file = []
for each in ner_data_sentenced:
  for a in each:
    to_file.append(a+"\n")
with open(ne_deneme_path, "w") as file1:
    file1.writelines(to_file)


# Reading Lexicon and Prepared Data

In [3]:
lexicon_person = []
lexicon_location = []
lexicon_organization = []


with open(path_per) as f:
  for line in f:
    lexicon_person.append(line[:-1])

with open(path_loc) as f:
  for line in f:
    lexicon_location.append(line[:-1])  

with open(path_org) as f:
  for line in f:
    lexicon_organization.append(line[:-1])


ner_data = []
with open(ne_deneme_path) as f:
    for line in f:
      ner_data.append(line[:-1])

ner_data_sentenced = []

line_cnt = 1
temp_line = []
for each in ner_data:
  current_line = int(each.split()[0])
  if current_line == line_cnt:
    temp_line.append(each)
  else:
    ner_data_sentenced.append(temp_line)
    temp_line = [each]
    line_cnt += 1
ner_data_sentenced.append(temp_line)

new_data = []

for i in range(len(ner_data_sentenced)):
  if i % 5 == 0:
    new_data.append(ner_data_sentenced[i])

for i in range(len(ner_data_sentenced)):
  if i % 5 == 1:
    new_data.append(ner_data_sentenced[i])

for i in range(len(ner_data_sentenced)):
  if i % 5 == 2:
    new_data.append(ner_data_sentenced[i])

for i in range(len(ner_data_sentenced)):
  if i % 5 == 3:
    new_data.append(ner_data_sentenced[i])

for i in range(len(ner_data_sentenced)):
  if i % 5 == 4:
    new_data.append(ner_data_sentenced[i])


# CRF function definations

In [4]:
def get_pos(line):
    tags = line.split()[2].split("^")[-1]
    tag = "0"
    if(len(tags.split("+")) > 1):
        tag = tags.split("+")[1]
    return tag

def get_root(line):
    tags = line.split()[2]
    tag = tags.split("+")[0]
    return tag

def get_prop(line):
    tag = False
    if "+Prop" in line:
        tag = True
    return tag

def get_noun_case(line):
    tags = line.split()[2]
    tag = tags.split("+")[-1]
    if tag not in ["Acc", "Dat", "Equ", "Gen", "Ins", "Nom", "Loc", "Abl"]:
        tag = "0"
    return tag

def get_orthographic_case(line):
    token = line.split()[1]
    tag = "LC"
    if token[0].isupper():
        tag = "UC"
    return tag

def get_all_inflectional(line):
    tags = line.split()[2].split("^")[-1]
    all_inf = ""
    if(len(tags.split("+")) > 1):
        tag = tags.split("+")[2:]
        all_inf = ' '.join(tag)
    else:
        all_inf = "0"
    return all_inf


def check_lexicon(line):
    token = line.split()[1]
    in_lexicon = 0
    if token[0].isupper() and (token in lexicon_person):
      in_lexicon = "PER"
    elif token[0].isupper() and (token in lexicon_location):
      in_lexicon = "LOC"
    elif token[0].isupper() and (token in lexicon_organization):
      in_lexicon = "ORG"
    return in_lexicon

# def check_person(line):
#   token = line.split()[1]
#   return (token[0].isupper() and (token in lexicon_person))


# def check_organization(line):
#   token = line.split()[1]
#   return (token[0].isupper() and (token in lexicon_organization))


# def check_location(line):
#   token = line.split()[1]
#   return (token[0].isupper() and (token in lexicon_location))



def word2features(sent, i):

    word = sent[i].split()[1]
    root = get_root(sent[i])
    postag = get_pos(sent[i])
    prop = get_prop(sent[i])
    noun_case = get_noun_case(sent[i])
    orthographic_case = get_orthographic_case(sent[i])
    all_inflectional = get_all_inflectional(sent[i])

    in_lexicon = check_lexicon(sent[i])

    # in_loc_lexicon = False
    # in_org_lexicon = False
    # in_per_lexicon = False

    # in_loc_lexicon = check_location(sent[i])
    # if not in_loc_lexicon:
    #   in_org_lexicon = check_organization(sent[i])
    # if not in_loc_lexicon and not in_org_lexicon:
    #   in_per_lexicon = check_person(sent[i])


    bos = False
    if i == 0:
      bos = True

    features = {
        'word.bias': 1.0,
        'word.root' : root,
        'word.postag': postag,
        'word.prop' : prop,
        'word.noun_case' : noun_case,
        'word.orthographic_case' : orthographic_case,
        'word.all_inflectional' : all_inflectional,
        'word.BOS' : bos,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit(),
        'word.isnotpunc()': word.isalpha(),
        'word.in_lexicon': in_lexicon,

        # 'word.in_loc_lexicon' : in_loc_lexicon,
        # 'word.in_org_lexicon' : in_org_lexicon,
        # 'word.in_per_lexicon' : in_per_lexicon,
    }
    if i > 0:
      root1 = get_root(sent[i-1])
      postag1 = get_pos(sent[i-1])
      prop1 = get_prop(sent[i-1])
      noun_case1 = get_noun_case(sent[i-1])
      orthographic_case1 = get_orthographic_case(sent[i-1])
      all_inflectional1 = get_all_inflectional(sent[i-1])
      word1 = sent[i-1].split()[1]
      in_lexicon1 = check_lexicon(sent[i-1])

      # in_loc_lexicon1 = False
      # in_org_lexicon1 = False
      # in_per_lexicon1 = False

      # in_loc_lexicon1 = check_location(sent[i-1])
      # if not in_loc_lexicon1:
      #   in_org_lexicon1 = check_organization(sent[i-1])
      # if not in_loc_lexicon1 and not in_org_lexicon1:
      #   in_per_lexicon1 = check_person(sent[i-1])

      features.update({
        '-1:word.root' : root1,
        '-1:word.postag': postag1,
        '-1:word.prop' : prop1,
        '-1:word.noun_case' : noun_case1,
        '-1:word.orthographic_case' : orthographic_case1,
        '-1:word.all_inflectional' : all_inflectional1,
        '-1:word.lower()': word1.lower(),
        '-1:word[-3:]': word1[-3:],
        '-1:word[-2:]': word1[-2:],
        '-1:word.isdigit()': word1.isdigit(),
        '-1:word.in_lexicon': in_lexicon1,
        # '-1:word.in_loc_lexicon' : in_loc_lexicon1,
        # '-1:word.in_org_lexicon' : in_org_lexicon1,
        # '-1:word.in_per_lexicon' : in_per_lexicon1,
      })
        
    if i < len(sent)-1:
      root1 = get_root(sent[i+1])
      postag1 = get_pos(sent[i+1])
      prop1 = get_prop(sent[i+1])
      noun_case1 = get_noun_case(sent[i+1])
      orthographic_case1 = get_orthographic_case(sent[i+1])
      all_inflectional1 = get_all_inflectional(sent[i+1])
      word1 = sent[i+1].split()[1]
      in_lexicon1 = check_lexicon(sent[i+1])

      # in_loc_lexicon1 = False
      # in_org_lexicon1 = False
      # in_per_lexicon1 = False

      # in_loc_lexicon1 = check_location(sent[i+1])
      # if not in_loc_lexicon1:
      #   in_org_lexicon1 = check_organization(sent[i+1])
      # if not in_loc_lexicon1 and not  in_org_lexicon1:
      #   in_per_lexicon1 = check_person(sent[i+1])

      features.update({
        '+1:word.root' : root1,
        '+1:word.postag': postag1,
        '+1:word.prop' : prop1,
        '+1:word.noun_case' : noun_case1,
        '+1:word.orthographic_case' : orthographic_case1,
        '+1:word.all_inflectional' : all_inflectional1,
        '+1:word.lower()': word1.lower(),
        '+1:word[-3:]': word1[-3:],
        '+1:word[-2:]': word1[-2:],
        '+1:word.isdigit()': word1.isdigit(),
        '+1:word.in_lexicon': in_lexicon1,
        # '+1:word.in_loc_lexicon' : in_loc_lexicon1,
        # '+1:word.in_org_lexicon' : in_org_lexicon1,
        # '+1:word.in_per_lexicon' : in_per_lexicon1,
      })
    else:
        features['word.EOS'] = True
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
  label_list = []
  for each in sent:
    label_list.append(each.split()[-1])
  return label_list

def sent2tokens(sent):
  token_list = []
  for each in sent:
    token_list.append(each.split()[1])
  return token_list

# sent2features(test_data[10])[0]
# test_data[10]
# sent2tokens(train_data[0])[0]
# sent2labels(train_data[0])[0]


def run_crf(fold):
  # data = copy.copy(new_data)

  temp_data = copy.copy(featurezed_data)
  temp_labels = copy.copy(featurezed_labels)


  X_test = temp_data[fold[0]:fold[1]]
  y_test = temp_labels[fold[0]:fold[1]]
  del temp_data[fold[0]:fold[1]]
  del temp_labels[fold[0]:fold[1]]

  X_train = temp_data
  y_train = temp_labels


  # test_data = data[fold[0]:fold[1]]
  # del data[fold[0]:fold[1]]
  # train_data = data

  # X_train = [sent2features(s) for s in train_data]
  # y_train = [sent2labels(s) for s in train_data]

  # X_test = [sent2features(s) for s in test_data]
  # y_test = [sent2labels(s) for s in test_data]

  crf = sklearn_crfsuite.CRF(
      algorithm='lbfgs', 
      c1=0.1, 
      c2=0.1, 
      max_iterations=100, 
      all_possible_transitions=True,
      # num_memories = 30
  )

  crf.fit(X_train, y_train)
  labels = list(crf.classes_)
  labels.remove('O')
  y_pred = crf.predict(X_test)

  sorted_labels = sorted(
      labels, 
      key=lambda name: (name[1:], name[0])
      )
  print("F1 Score for:" , fold , "is: " ,metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels))
  # print("Classification Report for:" , fold , "is: \n" ,metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))
  return metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3)

# Folds and CRF Runs

In [5]:
folds = [(0,2000), (2000,4000), (4000,6000), (6000,8000), (8000,10000)]
featurezed_data = [sent2features(s) for s in new_data]
featurezed_labels = [sent2labels(s) for s in new_data]
print("featurezation done")
reps =[]
for fold in folds:
  reps.append(run_crf(fold))

featurezation done
F1 Score for: (0, 2000) is:  0.910418113904667
F1 Score for: (2000, 4000) is:  0.9248508042918707
F1 Score for: (4000, 6000) is:  0.92677417853977
F1 Score for: (6000, 8000) is:  0.9133060665223309
F1 Score for: (8000, 10000) is:  0.9156499918978198


# Calculating Average and Outputs of the results

In [6]:
index_B_LOC_precision = 5
index_B_LOC_recall = 6
index_B_LOC_f1_score = 7

index_I_LOC_precision = 10
index_I_LOC_recall = 11
index_I_LOC_f1_score = 12


index_B_ORG_precision = 15
index_B_ORG_recall = 16
index_B_ORG_f1_score = 17

index_I_ORG_precision = 20
index_I_ORG_recall = 21
index_I_ORG_f1_score = 22


index_B_PER_precision = 25
index_B_PER_recall = 26
index_B_PER_f1_score = 27

index_I_PER_precision = 30
index_I_PER_recall = 31
index_I_PER_f1_score = 32


index_micro_avg_precision = 36
index_micro_avg_recall = 37
index_micro_avg_f1_score = 38


index_macro_avg_precision = 42
index_macro_avg_recall = 43
index_macro_avg_f1_score = 44


index_weighted_avg_precision = 48
index_weighted_avg_recall = 49
index_weighted_avg_f1_score = 50



total_B_LOC_precision = 0
total_B_LOC_recall = 0
total_B_LOC_f1_score = 0

total_I_LOC_precision = 0
total_I_LOC_recall = 0
total_I_LOC_f1_score = 0


total_B_ORG_precision = 0
total_B_ORG_recall = 0
total_B_ORG_f1_score = 0

total_I_ORG_precision = 0
total_I_ORG_recall = 0
total_I_ORG_f1_score = 0


total_B_PER_precision = 0
total_B_PER_recall = 0
total_B_PER_f1_score = 0

total_I_PER_precision = 0
total_I_PER_recall = 0
total_I_PER_f1_score = 0


total_micro_avg_precision = 0
total_micro_avg_recall = 0
total_micro_avg_f1_score = 0


total_macro_avg_precision = 0
total_macro_avg_recall = 0
total_macro_avg_f1_score = 0


total_weighted_avg_precision = 0
total_weighted_avg_recall = 0
total_weighted_avg_f1_score = 0

for each_fold in reps:

  total_B_LOC_precision += float(each_fold.split()[index_B_LOC_precision])
  total_B_LOC_recall += float(each_fold.split()[index_B_LOC_recall])
  total_B_LOC_f1_score += float(each_fold.split()[index_B_LOC_f1_score])

  total_I_LOC_precision += float(each_fold.split()[index_I_LOC_precision])
  total_I_LOC_recall += float(each_fold.split()[index_I_LOC_recall])
  total_I_LOC_f1_score += float(each_fold.split()[index_I_LOC_f1_score])


  total_B_ORG_precision += float(each_fold.split()[index_B_ORG_precision])
  total_B_ORG_recall += float(each_fold.split()[index_B_ORG_recall])
  total_B_ORG_f1_score += float(each_fold.split()[index_B_ORG_f1_score])

  total_I_ORG_precision += float(each_fold.split()[index_I_ORG_precision])
  total_I_ORG_recall += float(each_fold.split()[index_I_ORG_recall])
  total_I_ORG_f1_score += float(each_fold.split()[index_I_ORG_f1_score])


  total_B_PER_precision += float(each_fold.split()[index_B_PER_precision])
  total_B_PER_recall += float(each_fold.split()[index_B_PER_recall])
  total_B_PER_f1_score += float(each_fold.split()[index_B_PER_f1_score])

  total_I_PER_precision += float(each_fold.split()[index_I_PER_precision])
  total_I_PER_recall += float(each_fold.split()[index_I_PER_recall])
  total_I_PER_f1_score += float(each_fold.split()[index_I_PER_f1_score])


  total_micro_avg_precision += float(each_fold.split()[index_micro_avg_precision])
  total_micro_avg_recall += float(each_fold.split()[index_micro_avg_recall])
  total_micro_avg_f1_score += float(each_fold.split()[index_micro_avg_f1_score])


  total_macro_avg_precision += float(each_fold.split()[index_macro_avg_precision])
  total_macro_avg_recall += float(each_fold.split()[index_macro_avg_recall])
  total_macro_avg_f1_score += float(each_fold.split()[index_macro_avg_f1_score])


  total_weighted_avg_precision += float(each_fold.split()[index_weighted_avg_precision])
  total_weighted_avg_recall += float(each_fold.split()[index_weighted_avg_recall])
  total_weighted_avg_f1_score += float(each_fold.split()[index_weighted_avg_f1_score])


total_B_LOC_precision /= 5
total_B_LOC_recall /= 5
total_B_LOC_f1_score /= 5

total_I_LOC_precision /= 5
total_I_LOC_recall /= 5
total_I_LOC_f1_score /= 5


total_B_ORG_precision /= 5
total_B_ORG_recall /= 5
total_B_ORG_f1_score /= 5

total_I_ORG_precision /= 5
total_I_ORG_recall /= 5
total_I_ORG_f1_score /= 5


total_B_PER_precision /= 5
total_B_PER_recall /= 5
total_B_PER_f1_score /= 5

total_I_PER_precision /= 5
total_I_PER_recall /= 5
total_I_PER_f1_score /= 5


total_micro_avg_precision /= 5
total_micro_avg_recall /= 5
total_micro_avg_f1_score /= 5


total_macro_avg_precision /= 5
total_macro_avg_recall /= 5
total_macro_avg_f1_score /= 5


total_weighted_avg_precision /= 5
total_weighted_avg_recall /= 5
total_weighted_avg_f1_score /= 5


average_classification_report = "\t\tprecision\trecall\t\tf1-score\n\n" 
average_classification_report += "\tB-LOC\t" + str(total_B_LOC_precision)[:5] +"\t\t" + str(total_B_LOC_recall)[:5] + "\t\t" + str(total_B_LOC_f1_score)[:5] + "\n"
average_classification_report += "\tI_LOC\t" + str(total_I_LOC_precision)[:5] +"\t\t" + str(total_I_LOC_recall)[:5] + "\t\t" + str(total_I_LOC_f1_score)[:5] + "\n"

average_classification_report += "\tB-ORG\t" + str(total_B_ORG_precision)[:5] +"\t\t" + str(total_B_ORG_recall)[:5] + "\t\t" + str(total_B_ORG_f1_score)[:5] + "\n"
average_classification_report += "\tI_ORG\t" + str(total_I_ORG_precision)[:5] +"\t\t" + str(total_I_ORG_recall)[:5] + "\t\t" + str(total_I_ORG_f1_score)[:5] + "\n"

average_classification_report += "\tB-PER\t" + str(total_B_PER_precision)[:5] +"\t\t" + str(total_B_PER_recall)[:5] + "\t\t" + str(total_B_PER_f1_score)[:5] + "\n"
average_classification_report += "\tI_PER\t" + str(total_I_PER_precision)[:5] +"\t\t" + str(total_I_PER_recall)[:5] + "\t\t" + str(total_I_PER_f1_score)[:5] + "\n\n"

average_classification_report += "    micro avg\t" + str(total_micro_avg_precision)[:5] +"\t\t" + str(total_micro_avg_recall)[:5] + "\t\t" + str(total_micro_avg_f1_score)[:5] + "\n"
average_classification_report += "    macro avg\t" + str(total_macro_avg_precision)[:5] +"\t\t" + str(total_macro_avg_recall)[:5] + "\t\t" + str(total_macro_avg_f1_score)[:5] + "\n"
average_classification_report += " weighted avg\t" + str(total_weighted_avg_precision)[:5] +"\t\t" + str(total_weighted_avg_recall)[:5] + "\t\t" + str(total_weighted_avg_f1_score)[:5] + "\n"

print("Average --> (Fold 1 + Fold 2 + Fold 3 + Fold 4 + Fold 5) / 5")
print(average_classification_report, "\n<|--------------------------------------------------------|>\n\n")

for i in range(len(reps)):
  print("Fold" ,i, "-->" , folds[i])
  print(reps[i] , "\n<|--------------------------------------------------------|>\n\n")

Average --> (Fold 1 + Fold 2 + Fold 3 + Fold 4 + Fold 5) / 5
		precision	recall		f1-score

	B-LOC	0.949		0.937		0.943
	I_LOC	0.824		0.722		0.768
	B-ORG	0.923		0.912		0.917
	I_ORG	0.868		0.860		0.863
	B-PER	0.933		0.937		0.935
	I_PER	0.909		0.921		0.915

    micro avg	0.921		0.915		0.918
    macro avg	0.901		0.882		0.890
 weighted avg	0.921		0.915		0.918
 
<|--------------------------------------------------------|>


Fold 0 --> (0, 2000)
              precision    recall  f1-score   support

       B-LOC      0.945     0.938     0.942       850
       I-LOC      0.849     0.646     0.734       113
       B-ORG      0.909     0.902     0.905       650
       I-ORG      0.852     0.828     0.840       424
       B-PER      0.932     0.942     0.937      1055
       I-PER      0.905     0.910     0.908       458

   micro avg      0.916     0.906     0.911      3550
   macro avg      0.899     0.861     0.877      3550
weighted avg      0.915     0.906     0.910      3550
 
<|------------